# core

> Config package for python

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export

import yaml
import os
import pathlib
import inspect
import traceback

In [ ]:
#| exporti
def get_env(
        var:str, # The environment variable to get
        default:str = "default" # The default value to return if the environment variable is not set
    ):
    "Get a value from the environment variables."
    return os.environ.get(var, "default")


In [ ]:

assert get_env('TEST') == 'default'
os.environ['TEST'] = 'test'
assert get_env('TEST') == 'test'



In [ ]:
#| export
def config_get(
        value: str = None, # Name of value (None to read all values)
        py_config_active:str = None, # Name of configuration to read from. Defaults to the value of the R_CONFIG_ACTIVE environment variable ("default" if the variable does not exist).
        file:str = 'config.yaml', # Configuration file to read from (defaults to "config.yml"). If the file isn't found at the location specified then parent directories are searched for a file of the same name.
        encoding:str = 'utf-8'
    ):
    "Get a value from the `config.yaml` file.Read from the currently active configuration, retrieving either a single named value or all values as a list."
    if py_config_active is None:
        py_config_active = get_env('R_CONFIG_ACTIVE', 'default')
    
    sources = {
        inspect.stack()[0][1],
        os.path.dirname(traceback.extract_stack()[-2].filename),
        os.getcwd(),
        os.path.abspath(""),
    }

    file_exists = False
    for source in sources:
        filename = pathlib.Path(source, file)
        if os.path.exists(filename):
            file_exists = True
            # print(f"Found file {file} with source {source}")
            break

        else:
            # print(f"File {file} not found with source {source}")
            pass

    if not file_exists:
        source_locs = '\n - '.join(sources)
        raise FileNotFoundError(f"File {file} not found in any of these locations: \n - {source_locs}")


    dir_path = os.getcwd()
    file = pathlib.Path(dir_path, filename)

    with open(file, 'r') as stream:
        conf = yaml.safe_load(stream)
    
    if value is None:
        return conf[py_config_active]
    else:
        return conf[py_config_active][value]


In [ ]:
assert config_get('trials') == 5
assert config_get('trials', 'production') == 30



In [ ]:
test_fail(
    lambda: config_get('trials', file = "nofile.yaml"),
    contains = "File"
)


In [ ]:

config_get(py_config_active="production")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()